# 계층형 분류기 학습

본 프로젝트에서는 텍스트 데이터를 2단계에 걸쳐 분류하는 계층형 분류기(Hierarchical Classifier)를 구현합니다. 이 접근 방식은 대분류(그룹)를 먼저 판별한 후, 해당 그룹 내에서 세부 분류(서브레딧)를 수행하여 전체적인 분류 정확도와 효율성을 높이는 것을 목표로 합니다.

## 학습 과정

1.  **데이터 준비**: 
    *   **데이터 확인**: 데이터 형태와 결측값 개수 등을 확인합니다.
    *   **전처리**: 결측값 및 불용어 제거 등의 전처리를 수행합니다.
    *   **벡터화**: 전처리된 텍스트 데이터를 `TfidfVectorizer`를 사용하여 고차원 벡터로 변환합니다. 이를 통해 텍스트의 의미적 특징을 수치적으로 표현합니다.

2.  **상위 분류기 (Top-level Classifier) 학습**:
    *   **목표**: 텍스트가 3개의 그룹('Mystery', 'Dev', 'Culture') 중 어디에 속하는지 분류합니다.
    *   **모델**: 대용량 텍스트 데이터에 효율적인 `SGDClassifier`를 사용합니다.
    *   **최적화**: `RandomizedSearchCV`를 이용해 최적의 하이퍼파라미터를 탐색하여 모델의 일반화 성능을 극대화합니다.

3.  **하위 분류기 (Sub-level Classifiers) 학습**:
    *   **목표**: 각 그룹별로 3개의 서브레딧 중 어느 것인지 세부적으로 분류합니다.
    *   **모델**: 그룹별로 독립된 `SGDClassifier`를 학습시켜, 해당 그룹의 데이터 특성에 더 전문화된 모델을 만듭니다.
    *   **최적화**: 각 하위 분류기 역시 `RandomizedSearchCV`를 통해 개별적으로 최적화합니다.

4.  **모델 저장**: 학습이 완료된 상위 분류기와 3개의 하위 분류기 모델을 각각 파일로 저장한다.

In [12]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [13]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import re

In [14]:
FILE_PATH = 'assets/' 
MODEL_PATH = 'models/'

In [15]:
# 전처리에 nltk의 stopwords 활용
try:
    from nltk.corpus import stopwords
    english_stopwords = stopwords.words('english')
except ImportError:
    print("경고: nltk 라이브러리가 설치되지 않았거나 'stopwords'가 다운로드되지 않았습니다.")
    english_stopwords = set()

## 1. 데이터 준비

### 서브레딧 목록 (총 9개, 3그룹)

수집 데이터의 다양성을 확보하고 유사 범주별 관계성을 분석하기 위해 아래와 같이 3개 그룹, 총 9개의 서브레딧을 선정했습니다.

그룹 | 주제 | 대상 서브레딧
-- | -- | --
A | 미스터리 및 특이 현상 | r/Thetruthishere, r/Glitch_in_the_Matrix, r/UnresolvedMysteries
B | 개발 및 커리어 | r/learnprogramming, r/cscareerquestions, r/SideProject
C | 문화 및 심층 분석 | r/TrueFilm, r/booksuggestions, r/TrueGaming

In [16]:
# 서브레딧 및 그룹 정의
SUBREDDITS = [
    'Thetruthishere', 'Glitch_in_the_Matrix', 'UnresolvedMysteries',
    'learnprogramming', 'cscareerquestions', 'SideProject',
    'TrueFilm', 'booksuggestions', 'TrueGaming'
]
GROUP_MAP = {'Mystery': SUBREDDITS[0:3], 'Dev': SUBREDDITS[3:6], 'Culture': SUBREDDITS[6:9]}
VECTOR_DIMENSION = 5000  # 문서 벡터 차원
N_SELECT = 30   # 센트로이드 정제 시 선별할 상위 N개 문서

In [17]:
# 데이터 로드
df = pd.read_csv(FILE_PATH + 'reddit_posts.csv')

subreddit_df = df[df['subreddit'] == SUBREDDITS[0]].copy()
subreddit_df.head(5)

,subreddit,title,text
0,Thetruthishere,about truth Leviathan,about Leviathan isn't scary monster is actuall...
1,Thetruthishere,Weird nightmare last night,NaN
2,Thetruthishere,The streetlight in front of my house turns off...,"For the past month, the streetlight directly o..."
3,Thetruthishere,The reflection in the window was a few seconds...,"I was on a nearly empty late-night train, star..."
4,Thetruthishere,Strange knocking in my empty apartment at night.,I've been living alone in this old apartment f...


In [18]:
for subreddit in SUBREDDITS:
    subreddit_df = df[df['subreddit'] == subreddit]
    total_count = len(subreddit_df)
    nan_count = subreddit_df['text'].isna().sum()
    print(f"--- r/{subreddit} ---")
    print(f"전체 데이터 개수: {total_count}")
    print(f"text가 NaN인 데이터 개수: {nan_count}\n")

--- r/Thetruthishere ---
전체 데이터 개수: 995
text가 NaN인 데이터 개수: 45

--- r/Glitch_in_the_Matrix ---
전체 데이터 개수: 996
text가 NaN인 데이터 개수: 9

--- r/UnresolvedMysteries ---
전체 데이터 개수: 991
text가 NaN인 데이터 개수: 0

--- r/learnprogramming ---
전체 데이터 개수: 995
text가 NaN인 데이터 개수: 0

--- r/cscareerquestions ---
전체 데이터 개수: 987
text가 NaN인 데이터 개수: 0

--- r/SideProject ---
전체 데이터 개수: 999
text가 NaN인 데이터 개수: 53

--- r/TrueFilm ---
전체 데이터 개수: 988
text가 NaN인 데이터 개수: 0

--- r/booksuggestions ---
전체 데이터 개수: 1000
text가 NaN인 데이터 개수: 0

--- r/TrueGaming ---
전체 데이터 개수: 992
text가 NaN인 데이터 개수: 3



In [19]:
# 결측값 제거
print(f"총 결측값 개수: {df['text'].isna().sum()}")
df.dropna(subset=['text'], inplace=True)
print(f"결측값 제거 후 결측값 개수: {df['text'].isna().sum()}")  # 결측값이 제거되었는지 확인

총 결측값 개수: 110
결측값 제거 후 결측값 개수: 0


In [20]:
# 효율적인 불용어 처리를 위해 리스트를 집합(set)으로 변환
stopwords_set = set(english_stopwords)

def preprocess_text(text: str) -> str:
    """
    텍스트 데이터를 전처리하는 함수:
    1. 소문자 변환
    2. 알파벳과 공백을 제외한 모든 문자 제거
    3. 불용어 제거
    """
    # 입력값이 문자열이 아닌 경우 빈 문자열 반환
    if not isinstance(text, str):
        return np.nan
    
    # url 제거
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # 정규표현식을 사용하여 알파벳과 공백 외의 문자 제거 및 소문자 변환
    text = re.sub(r'[^a-zA-Z\s]', '', text).lower()
    
    # 공백을 기준으로 단어 토큰화 후 불용어 제거
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords_set]

    # 결과가 없으면(모두 불용어거나 특수문자였으면) NaN 반환
    if not filtered_words:
        return np.nan
    
    # 처리된 단어들을 다시 하나의 문자열로 결합
    return " ".join(filtered_words)

preprocessed_df = pd.read_csv(FILE_PATH + 'reddit_posts.csv')

# 'title'과 'text' 열을 결합하여 'content' 열 생성
preprocessed_df['content'] = preprocessed_df['title'] + " " + preprocessed_df['text']

# 'content' 열에 전처리 함수 적용하여 'processed_content' 열 생성
preprocessed_df['preprocessed_content'] = preprocessed_df['content'].apply(preprocess_text)

# 원본 'content', 'title', 'text' 열 제거
preprocessed_df.drop(columns=['content', 'title', 'text'], inplace=True)

# 전처리 후 결측값 제거
preprocessed_df.dropna(subset=['preprocessed_content'], inplace=True)

# 전처리 결과 저장
preprocessed_df.to_csv(FILE_PATH + 'reddit_posts_preprocessed.csv', index=False)

# 전처리 결과 확인
print('결측값 개수: ', preprocessed_df['preprocessed_content'].isna().sum())
preprocessed_df.head(5)

결측값 개수:  0


,subreddit,preprocessed_content
0,Thetruthishere,truth leviathan leviathan isnt scary monster a...
2,Thetruthishere,streetlight front house turns every time walk ...
3,Thetruthishere,reflection window seconds behind nearly empty ...
4,Thetruthishere,strange knocking empty apartment night ive liv...
5,Thetruthishere,whats unexplainable thing youve ever witnessed...


## 2. 상위 분류기 학습

In [21]:
# 상위 그룹 분류기 학습
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.stats import loguniform

# 벡터화
vectorizer = TfidfVectorizer(max_features=VECTOR_DIMENSION, ngram_range=(1, 2))
X = vectorizer.fit_transform(preprocessed_df['preprocessed_content']).toarray()

# 상위 그룹 라벨링 (서브레딧 -> 그룹 매핑)
reverse_group_map = {subreddit: group for group, subreddits in GROUP_MAP.items() for subreddit in subreddits}
y_group = preprocessed_df['subreddit'].map(reverse_group_map)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y_group, test_size=0.2, random_state=42, stratify=y_group
)

print(f"데이터 분할 완료: 학습용 {X_train.shape[0]}개, 테스트용 {X_test.shape[0]}개")


# RandomizedSearchCV 설정

# 파라미터 분포 정의
param_dist = {
    'loss': ['log_loss'],  # 로지스틱 회귀 (확률 출력용)
    'alpha': loguniform(1e-5, 1e-1), 
    'penalty': ['l2', 'l1', 'elasticnet'],
    'max_iter': [1000, 2000, 3000]
}

# 기본 모델 설정
base_model = SGDClassifier(random_state=42, class_weight='balanced', n_jobs=1)

# 교차 검증 전략 (학습 데이터 안에서만 수행)
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# RandomizedSearchCV 객체 생성
random_search_top = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_dist,
    n_iter=20,          
    cv=cv_strategy,     
    scoring='accuracy', 
    n_jobs=-1,          
    verbose=1,
    random_state=42
)

print("상위 분류기(Top-level Classifier) 최적화 시작 (RandomizedSearch)...")

random_search_top.fit(X_train, y_train)

# 결과 출력 및 최종 평가
print("\n" + "="*50)
print(f"최적의 하이퍼파라미터: {random_search_top.best_params_}")
print(f"교차 검증 최고 점수 (Best CV Score): {random_search_top.best_score_:.4f}")

# 최적의 모델 추출
top_classifier = random_search_top.best_estimator_

# 성능 평가
y_pred = top_classifier.predict(X_test)
final_accuracy = accuracy_score(y_test, y_pred)

print("-" * 50)
print(f"최종 테스트 정확도 (Test Accuracy): {final_accuracy:.4f}")
print("="*50)
print("\n[상세 분류 리포트]")
print(classification_report(y_test, y_pred))

데이터 분할 완료: 학습용 7065개, 테스트용 1767개
상위 분류기(Top-level Classifier) 최적화 시작 (RandomizedSearch)...
Fitting 5 folds for each of 20 candidates, totalling 100 fits

최적의 하이퍼파라미터: {'alpha': np.float64(0.00016480446427978953), 'loss': 'log_loss', 'max_iter': 2000, 'penalty': 'l2'}
교차 검증 최고 점수 (Best CV Score): 0.9837
--------------------------------------------------
최종 테스트 정확도 (Test Accuracy): 0.9870

[상세 분류 리포트]
              precision    recall  f1-score   support

     Culture       0.98      0.98      0.98       595
         Dev       0.99      0.99      0.99       586
     Mystery       0.99      0.99      0.99       586

    accuracy                           0.99      1767
   macro avg       0.99      0.99      0.99      1767
weighted avg       0.99      0.99      0.99      1767



## 3. 하위 분류기 학습

In [22]:
# 세부 레딧 분류기 학습
# 그룹별로 별도의 분류기를 저장할 딕셔너리
sub_classifiers = {}
sub_best_params = {}

# 최적화 탐색 범위 설정 (RandomizedSearch용)
param_dist = {
    'loss': ['log_loss'], 
    'alpha': loguniform(1e-5, 1e-1), # 0.00001 ~ 0.1 사이 랜덤 탐색
    'penalty': ['l2', 'l1', 'elasticnet'],
    'max_iter': [1000, 2000]
}

print("하위 분류기 최적화 및 학습 시작...\n")

y_pred_sub = [] # 하위 분류기 예측값 저장 리스트
for group_name, sub_list in GROUP_MAP.items():
    print(f"--- [{group_name}] 그룹 전용 분류기 최적화 중 ---")
    
    # 해당 그룹 데이터 추출
    group_mask = preprocessed_df['subreddit'].isin(sub_list)
    
    X_group = X[group_mask.values]
    y_group_sub = preprocessed_df.loc[group_mask, 'subreddit']
    
    # 데이터 분할
    X_g_train, X_g_test, y_g_train, y_g_test = train_test_split(
        X_group, y_group_sub, test_size=0.2, random_state=42, stratify=y_group_sub
    )
    
    # RandomizedSearchCV 설정
    base_model = SGDClassifier(random_state=42, class_weight='balanced', n_jobs=1)
    
    search = RandomizedSearchCV(
        estimator=base_model,
        param_distributions=param_dist,
        n_iter=10,          # 그룹당 10번 탐색
        cv=3,               # 그룹별 3-Fold 사용
        scoring='accuracy',
        n_jobs=-1,
        random_state=42,
        verbose=0
    )
    
    # 최적화 실행
    search.fit(X_g_train, y_g_train)
    
    # 최적 모델 저장 및 평가
    best_model = search.best_estimator_
    sub_classifiers[group_name] = best_model
    sub_best_params[group_name] = search.best_params_
    
    # 검증 데이터로 성능 확인
    y_g_pred = best_model.predict(X_g_test)
    y_pred_sub.append(y_g_pred[0])
    acc = accuracy_score(y_g_test, y_g_pred)
    
    print(f"  -> 최적 설정: alpha={search.best_params_['alpha']:.5f}, penalty={search.best_params_['penalty']}")
    print(f"  -> 테스트 정확도: {acc:.4f}")

print("\n 모든 하위 분류기 학습 완료.")

하위 분류기 최적화 및 학습 시작...

--- [Mystery] 그룹 전용 분류기 최적화 중 ---
  -> 최적 설정: alpha=0.00007, penalty=l2
  -> 테스트 정확도: 0.9096
--- [Dev] 그룹 전용 분류기 최적화 중 ---
  -> 최적 설정: alpha=0.00031, penalty=elasticnet
  -> 테스트 정확도: 0.8754
--- [Culture] 그룹 전용 분류기 최적화 중 ---
  -> 최적 설정: alpha=0.00001, penalty=l1
  -> 테스트 정확도: 0.9866

 모든 하위 분류기 학습 완료.


## 4. 모델 저장

In [23]:
# 모델 저장
import joblib

# 벡터라이저 저장
joblib.dump(vectorizer, MODEL_PATH + 'tfidf_vectorizer.pkl')

# 상위 분류기 모델 저장
best_top_classifier = random_search_top.best_estimator_
joblib.dump(best_top_classifier, MODEL_PATH + 'top_classifier.pkl')

# 하위 분류기 모델 저장
for group_name, model in sub_classifiers.items():
    joblib.dump(model, MODEL_PATH + f'sub_classifier_{group_name}.pkl')